# 8 - Results

## Import Library

In [1]:
import joblib

import pandas as pd

optimal = "k_15_a_250_e_0.001"

## Import Model

In [2]:
model = joblib.load("outputs/models/btm/btm_{}.sav".format(optimal))

## Topic Proportion of Corpus

In [3]:
tpoc = pd.DataFrame(data = {"k" : [i + 1 for i in range(len(model.theta_))], "topic_proportion" : model.theta_})

# tpoc.to_csv("outputs/tpoc.csv", index = False)

## Topic Proportion of Document

### Population

In [4]:
import bitermplus as btm

df = pd.read_csv("inputs/tweets_3.csv")

texts = df["content_10"].str.strip().tolist()

X, vocabulary, vocab_dict = btm.get_words_freqs(texts)

docs_vec = btm.get_vectorized_docs(texts, vocabulary)

biterms = btm.get_biterms(docs_vec)

In [5]:
p_zd = pd.DataFrame(data = model.transform(docs_vec)).T

# p_zd.to_csv("outputs/tpod/p_zd_{}_population.csv".format(optimal))

100%|██████████| 236956/236956 [00:02<00:00, 82256.68it/s] 


In [6]:
df = pd.read_csv("inputs/tweets_3.csv")

p_zd = pd.read_csv("outputs/tpod/p_zd_{}_population.csv".format(optimal), index_col = 0)

p_zd = p_zd.T

p_zd.reset_index(drop = True, inplace = True)

p_zd.columns = ["topic_" + str(i + 1) for i in range(len(p_zd.columns))]

p_zd["topic_k"] = p_zd.idxmax(axis = 1)

tpod = df.merge(p_zd, left_index = True, right_index = True)

# tpod.to_csv("outputs/tpod/tpod_population.csv", index = False)

# tpod.to_excel("outputs/tpod/tpod_population.xlsx", index = False)

### Sample

In [7]:
p_zd = pd.read_csv("outputs/btm/p_zd_{}.csv".format(optimal), index_col = 0)

# p_zd.to_csv("outputs/tpod/p_zd_{}_sample.csv".format(optimal)) #

In [8]:
df = pd.read_csv("inputs/tweets_sample.csv") #

p_zd = pd.read_csv("outputs/tpod/p_zd_{}_sample.csv".format(optimal), index_col = 0)

p_zd = p_zd.T

p_zd.reset_index(drop = True, inplace = True)

p_zd.columns = ["topic_" + str(i + 1) for i in range(len(p_zd.columns))]

p_zd["topic_k"] = p_zd.idxmax(axis = 1)

tpod = df.merge(p_zd, left_index = True, right_index = True)

# tpod.to_csv("outputs/tpod/tpod_sample.csv", index = False)

# tpod.to_excel("outputs/tpod/tpod_sample.xlsx", index = False)

## Topic Proportion of Document (Sample)

In [9]:
tpod_sample = tpod.groupby("topic_k").sample(n = 100, random_state = 100)

# tpod_sample.to_csv("outputs/tpod/tpod_sample_sample.csv", index = False)

# tpod_sample.to_excel("outputs/tpod/tpod_sample_sample.xlsx", index = False)

## Top Words

In [10]:
p_wz = pd.read_csv("outputs/btm/p_wz_{}.csv".format(optimal), index_col = 0).T

dictionary = dict()

for i in p_wz.columns:
    dictionary["Topic " + str(i + 1)] = list(p_wz.sort_values(by = i, ascending = False).head(30).index)
    
top_words = pd.DataFrame.from_dict(dictionary)

# top_words.to_csv("outputs/top_words.csv", index = False)